In [1]:
!pip install tashaphyne
!pip install qalsadi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickledb: filename=pickleDB-0.9.2-py3-none-any.whl size=4254 sha256=ab0d8ee6bc1f3de4aacccfaec913add9adf75897b3502dfd23fed6b978427a95
  Stored in directory: /root/.c

In [2]:
# Add environment Packages paths to conda
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.simplefilter("ignore")

# Text preprocessing packages
import nltk # Text libarary
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import string # Removing special characters {#, @, ...}
import re # Regex Package
import regex
# Corpora is a group presenting multiple collections of text documents. A single collection is called corpus.
from nltk.corpus import stopwords # Stopwords
from tashaphyne.stemming import ArabicLightStemmer
from qalsadi.lemmatizer import Lemmatizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer # Stemmer & Lemmatizer
#from gensim.utils import simple_preprocess  # Text ==> List of Tokens

# Visualization Packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.3)
%matplotlib
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

Using matplotlib backend: <object object at 0x79ba9d907670>


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
class TextPreprocessing:

    def __init__(self):
        self.arabic_stopwords = set(stopwords.words("arabic"))

    def remove_stop_words(self, texts_tensor):
        # Define a function to remove stopwords from a single text
        def remove_stopwords_from_text(text):
            # Split the text into words
            words = tf.strings.split(text)
            # Check membership of each word in the set of stopwords
            mask = tf.logical_not(tf.reduce_any(tf.equal(words[:, tf.newaxis], list(self.arabic_stopwords)), axis=1))
            # Filter out stopwords
            filtered_words = tf.boolean_mask(words, mask)
            # Join the filtered words back into a single string
            return tf.strings.reduce_join(filtered_words, separator=' ', axis=-1)
        # Apply the function to each text in the tensor
        texts_without_stopwords = tf.map_fn(remove_stopwords_from_text, texts_tensor, dtype=tf.string)
        return texts_without_stopwords

    def Arabic_Lemmatizer(self, text):
        # Initialize WordNetLemmatizer
        lemmer = Lemmatizer()
        # Convert the TensorFlow tensor to a NumPy array
        text_array = text.numpy()
        # Tokenize each text in the array and lemmatize each word
        lemmatized_texts = []
        for t in text_array:
            tokens = nltk.word_tokenize(t.decode('utf-8'))  # Decode bytes to string
            lemmatized_words = [lemmer.lemmatize(word) for word in tokens]
            lemmatized_texts.append(" ".join(lemmatized_words))
        # Convert the list of lemmatized texts back to a TensorFlow tensor
        lemmatized_tensor = tf.constant(lemmatized_texts)
        return lemmatized_tensor

    def normalizeArabic(self, text):
        text = tf.strings.strip(text)
        text = tf.strings.regex_replace(text, "ى", "ي")
        text = tf.strings.regex_replace(text, "ؤ", "ء")
        text = tf.strings.regex_replace(text, "ئ", "ء")
        text = tf.strings.regex_replace(text, "ة", "ه")
        text = tf.strings.regex_replace(text, "[إأٱآا]", "ا")
        text = tf.strings.regex_replace(text, "وو", "و")
        text = tf.strings.regex_replace(text, "ييي", "ي")
        text = tf.strings.regex_replace(text, "يي", "ي")
        text = tf.strings.regex_replace(text, "اا", "ا")
        return text

    def Removing_non_arabic(self, text):
        return tf.strings.regex_replace(text, "[A-Za-z]+", " ")

    def Removing_numbers(self,text):
        return tf.strings.regex_replace(text, "[0-9]+", " ")

    def Removing_punctuations(self, text):
        # Define the set of punctuations to remove
        punctuations = """!"#$%&'()*+,،—-./:;<=>؟?@[\]^_`{|}~"""
        # Replace punctuations with space
        for char in punctuations:
            text = tf.strings.regex_replace(text, re.escape(char), " ")
        # Replace Arabic semicolon with empty string
        text = tf.strings.regex_replace(text, "؛", "")
        # Remove extra whitespace
        text = tf.strings.regex_replace(text, r"\s+", " ")
        # Strip leading and trailing whitespace
        text = tf.strings.strip(text)
        return text

    def Removing_urls(self, text):
        # Define the regex pattern for URLs
        url_pattern = r'https?://\S+|www\.\S+'
        # Replace URLs with an empty string
        text = tf.strings.regex_replace(text, url_pattern, "")
        return text


    def remove_emoji(self, text):
        emoji_pattern = tf.strings.regex_replace(
            text,
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]",
            ""
        )
        return tf.strings.strip(emoji_pattern)

    def remove_emoticons(self, text):
        # Define the emoticons you want to remove
        EMOTICONS = [':)', ':D', ':(', ':|', ';)', 'B)', ':@', '8)']
        # Replace emoticons with an empty string
        for emoticon in EMOTICONS:
            text = tf.strings.regex_replace(text, re.escape(emoticon), '')
        return text


    def remove_extra_Space(self,text):
        ## remove extra whitespace
        text = re.sub('\s+', ' ', text)
        return  " ".join(text.split())

    def remove_small_sentences(self,text):
        for i in range(len(df)):
            if len(df.text.iloc[i].split()) < 3:
                df.text.iloc[i] = np.nan

    def remove_hashtags_and_mentions(self, text):
        # Remove mentions
        text = tf.strings.regex_replace(text, r"@[\w]+", "")
        # Remove hashtags
        text = tf.strings.regex_replace(text, r"#[\w]+", "")
        return text

    def remove_special_characters(self, text):
        # Remove special characters like <<text>> and "text":
        text = tf.strings.regex_replace(text, r'<<"([^"]*)">>', r'\1')
        text = tf.strings.regex_replace(text, r'"([^"]*)"', r'\1')
        return text


In [ ]:
class Preprocessing:
    def __init__(self):
        self.text_preprocessor = TextPreprocessing()

    def preprocess_text(self, text):
        # Example: Apply all preprocessing steps
        text = self.text_preprocessor.remove_stop_words(text)
        text = self.text_preprocessor.remove_hashtags_and_mentions(text)
        text = self.text_preprocessor.remove_special_characters(text)
        text = self.text_preprocessor.remove_emoticons(text)
        text = self.text_preprocessor.Removing_urls(text)
        text = self.text_preprocessor.Removing_non_arabic(text)
        text = self.text_preprocessor.Removing_numbers(text)
        text = self.text_preprocessor.Removing_punctuations(text)
        text = self.text_preprocessor.remove_emoji(text)
        text = self.text_preprocessor.normalizeArabic(text)
        text = self.text_preprocessor.Arabic_Lemmatizer(text)
        return text


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA/culture.csv', index_col=0)

In [ ]:
tpp = Preprocessing()
text = tf.constant(df['text'].values)

In [ ]:
result = tpp.preprocess_text(text)

In [ ]:
decoded_strings = [string.numpy().decode('utf-8') for string in result]

In [ ]:
text[7].numpy().decode('utf-8')

' تاريخ الكتب زمن الحرب.. من اليونان القديمة حتى حرب إسرائيل على غزة في طليعة العام 1939، ومع حلول ظلام الحرب العالمية الثانية، كانت مكتبات لندن تضج بالزوار. وكان الفضول والرغبة في فهم آليات الحرب والأمم المتصارعة شغل الجميع الشاغل، بالتوازي مع الأمل والتوق للمعرفة. وعندما أجبرت القنابل الآلاف من سكان لندن على الاحتماء في محطات قطارات الأنفاق بالمدينة، غالبًا ما كان الناس يلجؤون لإنشاء مكتبات صغيرة لدعم تماسكهم معنويا. وتُظهر إحدى أشهر الصور الفوتوغرافية -التي التقطت في لندن زمن الحرب- مجموعة من الرجال الهادئين، يرتدون القبعات، ويتفحصون الكتب الموجودة على الرفوف في مكتبة قصر كنسينغتون التي تعرضت للقصف. ونشرت مجلة "نيويوركر" الأميركية تقريراً للكاتبة كلوديا روث بيربونت تلقي الضوء فيه على كيف أن الصراعات المسلحة فتحت شهية القراء على مصراعيها للكتب التي تتحدث عن الحروب، معززة مبيعات القصص الأميركية حول الصراعات السابقة. وتناول التقرير كذلك تاريخ الكتاب منذ القدم، مستعرضا تأثير الإلياذة ومكتبة الإسكندرية كمنارات للمعرفة والثقافة. كما تطرق إلى دور الكتب في تشكيل الهويات الثقافية والوطنية، م

In [ ]:
decoded_strings[7]

'تاريخ كتب زمن حرب يونان القديمه حرب اسراءيل غاز طليعه عام مع حلول ظلام حرب العالميه الثانيه كانت مكتب أدان ضاج زوار كان فضول والرغبه هم ال حرب والامم المتصارعه شغل جميع شاغل توازي الامل والتوق للمعرفه عندما اجبرت قنابل الالاف سكان أدان احتماء محط قطارة الانفاق بالمدينه غالب ناس يلجءون لانشاء مكتب صغير دعم تماسك معنوي ظهر شهر صور الفوتوغرافيه التي التقط أدان زمن حرب مجموع رجال الهادءين ارتد قبع تفحص كتب الموجوده رفوف مكتب قصر كنسينغتون تعرض قصف نشر مجله نيويوركر الاميركيه تقرير للكاتبه كلود روث بيربونت ألقى ضوء صراع المسلحه تحت شهي قراء مصراع كتب تحدث حروب معزز مبيع قصص الاميركيه حول صراع السابقه تناول تقرير تاريخ كتاب قدم مستعرض تاثير الالياذه مكتب الاسكندريه منار للمعرفه والثقافه تطرق دور كتب تشكيل هوة الثقافيه والوطنيه الاشاره الاثر مدمر حروب مكتب اوكرانيا غاز مءكدا الفكره الراسخه بان كتب يم تكون ادوات سلام تفاهم الانساني وجد الاذهان الراغبه استيعاب تطبيق مبادءها كتاب حرب في كتاب كتاب حرب استل كاتب بريطاني اندرو بيتيجري احداث حرب الاهليه الاميركيه مسلط ضوء كوخ عم توم الروايه الراءده

In [ ]:
tokenizer = Tokenizer(num_words=3000, oov_token="<unk>")

In [ ]:
tokenizer.fit_on_texts(decoded_strings)

In [ ]:
tokenizer.word_index

{'<unk>': 1,
 'عام': 2,
 'في': 3,
 'كتاب': 4,
 'عالم': 5,
 'تاريخ': 6,
 'م': 7,
 'عرب': 8,
 'قد': 9,
 'ه': 10,
 '‘': 11,
 'العربيه': 12,
 'كانت': 13,
 'ال': 14,
 'قال': 15,
 'نهو': 16,
 'خلال': 17,
 'ت': 18,
 'بن': 19,
 'شعر': 20,
 'يوم': 21,
 'عمل': 22,
 'ابن': 23,
 'أقول': 24,
 'ثقاف': 25,
 'كان': 26,
 'كتب': 27,
 'من': 28,
 'نفس': 29,
 'كبير': 30,
 'كثير': 31,
 'ذلك': 32,
 'مصر': 33,
 'كاتب': 34,
 'قرن': 35,
 'شكل': 36,
 'عبد': 37,
 'دين': 38,
 'حرب': 39,
 'جامع': 40,
 'كل': 41,
 'على': 42,
 'عن': 43,
 'هي': 44,
 'نشر': 45,
 'عبر': 46,
 'جديد': 47,
 'الله': 48,
 'بان': 49,
 'دور': 50,
 'علي': 51,
 'حي': 52,
 'فن': 53,
 'ذكر': 54,
 'علم': 55,
 'تب': 56,
 'حديث': 57,
 'دول': 58,
 'مجتمع': 59,
 'حال': 60,
 'هذا': 61,
 'شاعر': 62,
 'شخص': 63,
 'غير': 64,
 'محمد': 65,
 'مسلم': 66,
 'يم': 67,
 'قدم': 68,
 'حول': 69,
 'سن': 70,
 'مع': 71,
 'اللغه': 72,
 'لان': 73,
 'الروايه': 74,
 'مدين': 75,
 'رغم': 76,
 'تراث': 77,
 'بلاد': 78,
 'بعد': 79,
 'حسب': 80,
 'عرف': 81,
 'ناس': 82,
 'لم': 83,
 